<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prédiction-de-la-valeur-foncière-d'un-bien-immobilier-à-Paris" data-toc-modified-id="Prédiction-de-la-valeur-foncière-d'un-bien-immobilier-à-Paris-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prédiction de la valeur foncière d'un bien immobilier à Paris</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Analyse-de-la-base-de-données-DVF" data-toc-modified-id="Analyse-de-la-base-de-données-DVF-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Analyse de la base de données DVF</a></span></li><li><span><a href="#Webscraping-de-ville-idéale" data-toc-modified-id="Webscraping-de-ville-idéale-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Webscraping de ville-idéale</a></span></li><li><span><a href="#Récupération-et-analyse-de-la-base-de-données-Filosofi" data-toc-modified-id="Récupération-et-analyse-de-la-base-de-données-Filosofi-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Récupération et analyse de la base de données Filosofi</a></span></li><li><span><a href="#Lien-entre-la-position-d'un-bien-dans-la-base-DVF-et-le-carreau-auquel-il-appartient-dans-la-base-Filosofi" data-toc-modified-id="Lien-entre-la-position-d'un-bien-dans-la-base-DVF-et-le-carreau-auquel-il-appartient-dans-la-base-Filosofi-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Lien entre la position d'un bien dans la base DVF et le carreau auquel il appartient dans la base Filosofi</a></span></li><li><span><a href="#Récupération-et-utilisation-de-la-base-de-données-BPE" data-toc-modified-id="Récupération-et-utilisation-de-la-base-de-données-BPE-1.0.5"><span class="toc-item-num">1.0.5&nbsp;&nbsp;</span>Récupération et utilisation de la base de données BPE</a></span></li><li><span><a href="#Analyse-et-mise-en-forme-avant-la-modélisation" data-toc-modified-id="Analyse-et-mise-en-forme-avant-la-modélisation-1.0.6"><span class="toc-item-num">1.0.6&nbsp;&nbsp;</span>Analyse et mise en forme avant la modélisation</a></span></li><li><span><a href="#Modélisation" data-toc-modified-id="Modélisation-1.0.7"><span class="toc-item-num">1.0.7&nbsp;&nbsp;</span>Modélisation</a></span><ul class="toc-item"><li><span><a href="#SVR-(support-vector-regression)" data-toc-modified-id="SVR-(support-vector-regression)-1.0.7.1"><span class="toc-item-num">1.0.7.1&nbsp;&nbsp;</span>SVR (support vector regression)</a></span></li><li><span><a href="#Forêt-aléatoire" data-toc-modified-id="Forêt-aléatoire-1.0.7.2"><span class="toc-item-num">1.0.7.2&nbsp;&nbsp;</span>Forêt aléatoire</a></span></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-1.0.7.3"><span class="toc-item-num">1.0.7.3&nbsp;&nbsp;</span>XGBoost</a></span></li></ul></li></ul></li></ul></li></ul></div>

Avant tout, pour exécuter tous les notebook, il est nécessaire [d'extraire le fichier good_data de notre drive](https://drive.google.com/drive/folders/1jNVEkmhRgsNgsXHlyKPRbNdJiBnFMHr_?usp=sharing) et ensuite de le déposer à la racine du projet (c'est à dire que le dossier good_data doit se retrouver à la même place que le dossier notebook, img, script etc.).

# Prédiction de la valeur foncière d'un bien immobilier à Paris

Dans ce projet, nous avons pour but de prédire la valeur foncière d'un bien immobilier à Paris. Pour ce faire, nous allons utiliser des données disponibles en opendata sur deux sites et nous allons faire du scraping pour obtenir une autre base de données.

### Analyse de la base de données DVF

Dans un premier temps, nous avons analysé la base DVF que l'on peut retrouver [ici](https://cerema.app.box.com/v/dvfplus-opendata). Dans cette table, on retrouve l'ensemble des transactions de biens immobiliers qui ont fait l'objet d'un acte notarié. De plus, on y retrouve certaines informations sur la date de la transactions mais aussi sur les caractéristiques des biens. Enfin, par soucis de ressource mémoire, nous nous sommes concentrés sur Paris.


Avec une première analyse sur le type du bien et la nature de la mutation, nous avons décidé de ne garder que les observations concernant les **ventes** de **maisons** ou d'**appartements**. En effet, la base DVF contient également des informations sur d'autres types de transactions (transferts de parcelles par exemple).

Nous avons alors étudié les variables. Nous nous sommes rendus compte que certaines variables prenaient très souvent la même valeur. Nous avons décidé de supprimer ces variables. Puis, nous avons étudié les valeurs manquantes de la variable concernant la valeur foncière. Leur fréquence étant très faible, nous avons décidé de les enlever du dataset. 

Après analyse des variables restantes, nous nous sommes rendu compte que la moyenne des valeurs foncières par année est en croissance de 2016 à 2019. Cette évolution temporelle est directement imputable à l'inflation immobilière à Paris (une revue de la littérature nous a permis d'obtenir quelques estimations). Nous souhaitons prédire la valeur d'un bien à partir de ses caractéristiques sans prendre en compte l'augmentation des prix liée à l'inflation. Nous avons donc considéré la composante stationnaire comme variable à expliquer.

Nous avons ensuite pris en main les différents datasets geopandas en les concaténant et en faisant une rapide analyse.

Toutes ces étapes se trouve [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/analysis_and_treat.ipynb)

### Webscraping de ville-idéale

[ville-idéale](https://www.ville-ideale.fr/) est un site internet sur lequel des internautes peuvent noter leur quartier sur plusieurs critères (santé, culture, sécurité...). Nous avons trouvé intéressant d'utiliser ses données pour de la visualisation. C'est pourquoi nous avons fait du webscraping pour récupérer les données pour les quartiers de Paris. Cependant, nous n'utiliserons pas ces données dans la modélisation car nous pensons que la variable "arrondissement" apportera les mêmes informations aux modèles.

Ce webscraping est effectué dans ce [script](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/script/ville_ideale_scraping.py)

### Récupération et analyse de la base de données Filosofi

Nous nous sommes alors penchés sur une autre base de données : Filosofi. Cette base de données est en opendata sur le [site](https://www.insee.fr/fr/statistiques/4176290?sommaire=4176305) de l'INSEE. Elle est notamment constituée d'indicateur de niveau de vie, d'inégalité et de pauvreté. Nous avons uniquement récupéré les données pour Paris. Les données sont disponibles à une échelle infracommunale sous la forme de carreaux de 200 mètres.

Nous avons analysé cette table dans sa globalité puis par carreaux.

Le notebook correspondant est [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/analyse_filosofi.ipynb).

### Lien entre la position d'un bien dans la base DVF et le carreau auquel il appartient dans la base Filosofi

Il a ensuite fallu associer à chaque mutation de la base DVF un carreau associé de la base Filosofi. Ceci permet de faire la jointure entres les deux tables. 

Pour ce faire, nous avons associé à chaque position de bien immobilier, le carreau auquel il appartient. Ce traitement était chronophage : nous sommes passés par Google Colab.

Voici le [notebook](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/jointure_mutation_filosofi.ipynb) associé.

### Récupération et utilisation de la base de données BPE

La base BPE (base permanente des équipements) contient des informations sur le niveau d'équipements et de services rendus à la population sur un territoire. Il est disponible sur le [site](https://www.insee.fr/fr/statistiques/3568656) de l'INSEE. Il y a plus de 130 types d'équipements (crèche, boulangerie, ...). 

Nous nous sommes évidemment restreints aux données parisiennes. Ces données sont, pour la majorité, géolocalisées.

Nous avons utilisé ces données pour calculer la distance à l'équipement le plus proche, pour chaque mutation et pour chaque type d'équipements. Nous avons retiré des équipements (qui semblaient peu influant dans le prix d'un bien immobilier) et fait des regroupements d'équipements.

Ces étapes sont disponibles dans ce [notebook](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/localisation_service_bpe.ipynb)

### Analyse et mise en forme avant la modélisation

Nous avons maintenant une base de données fusionnant des informations des quatres bases de données citées précédemment.

Nous avons alors essayé de corriger l'inflation. Pour ce faire, nous avons fait deux hypothèses :  
- la variation de la moyenne annuelle du prix des biens immobiliers à Paris est uniquement due à l'inflation.
- la moyenne annuelle de la valeur foncière est linéaire par rapport à l'année de la mutation entre 2014 et 2016 ainsi qu'entre 2016 et 2019

Nous avons alors pu corriger cette inflation.

Nous nous sommes ensuite penchés sur les valeurs manquantes provenant des deux dernières bases de données (Filosofi et BPE). Là encore, les observations avec des données manquantes étant très rares, nous avons décidé de les supprimer.

Puis, nous avons étudié les valeurs extrêmes de nos variables. Nous avons notamment remarqué que les maisons représentaient une grande part des valeurs extrêmes supérieures de la valeur foncière. De plus, c'est une très faible proportion de nos données. Nous avons donc à cette étape décidé de ne garder que les **appartements**. Enfin, nous avons retiré les observations avec une valeur foncière supérieur à 10 millions.

Enfin, nous avons fait le découpage de notre base de données en trois : sous-ensemble d'entraînement (60%), sous-ensemble de test (20%) et sous-ensemble de validation (20%).

Le notebook correspondant est [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/analyse_mise_en_forme_avant_modelisation.ipynb).

### Modélisation

#### SVR (support vector regression)

Le SVR (support vector regression) est une extension du SVM (support vector machine) dans le cadre d'une régression. Nous l'avons choisi pour tester un modèle autre que ceux de la classe ensembliste mais qui soit suffisamment complexe pour permettre du fine-tuning. De plus, nous avons choisi de faire une SVR avec noyau gaussien car il s'agit d'un noyau couramment utilisé. Enfin, nous n'avons pas fine-tuner le noyau par soucis de temps. 

Le notebook correspondant à cette modélisation est [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/modelisation_svr.ipynb)

#### Forêt aléatoire

Les forêts aléatoires sont une famille de modèles d'apprentissage supervisé faisant partie des méthodes de *bagging*. L'idée est de construire un nombre *B* d'arbres (typiquement CART) puis de les agréger (moyenne ou vote dominant). Chaque arbre est construit à l'aide d'un échantillon boostrap des données. De plus, à chaque partition, seules quelques variables sont testées pour trouver la partition optimale (tirage boostrap).

L'un des avantages des forêts aléatoires est de proposer des résultats respectables sans demander énormément d'adaptation des paramètres (*fine-tuning*).

Toute la modélisation est disponible [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/modelisation_rf.ipynb).

#### XGBoost

Le XGBoost et son petit frère le LightGBM sont à l'état de l'art pour ce genre de données tabulaires. Nommé le kaggle killer (aka XGBoost), c'est pour nous l'occassion d'éprouver ce modèle dans le cadre de notre problématique de prédiction du prix, et de comparer sa performance par rapport aux deux modèles précédents. Toute la modélisation est disponible [ici](https://github.com/Orlogskapten/dvf_ensae_sbra/blob/master/notebook/modelisation_wen.ipynb).
